### Fine-Tuning LLaMA-3.2 3B Instruct

This code will fine-tune the `LLaMA-3.2-3B-Instruct` model on the CounselChat dataset

In [1]:
import os

In [2]:
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [3]:
import torch
import pickle
from unsloth import FastLanguageModel, is_bfloat16_supported, train_on_responses_only
from transformers import TrainingArguments, DataCollatorForSeq2Seq, TextStreamer
from datasets import Dataset, DatasetDict
from trl import SFTTrainer

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


/data/mn27889/miniconda3/envs/mental-health-agents/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


🦥 Unsloth Zoo will now patch everything to make training faster!


Some links to be used for fine-tuning
1. https://www.kdnuggets.com/fine-tuning-llama-using-unsloth
2. https://www.analyticsvidhya.com/blog/2024/12/fine-tuning-llama-3-2-3b-for-rag/
3. https://www.linkedin.com/pulse/step-guide-use-fine-tune-llama-32-dr-oualid-soula-xmnff/
4. https://medium.com/@alexandros_chariton/how-to-fine-tune-llama-3-2-instruct-on-your-own-data-a-detailed-guide-e5f522f397d7 (this seems faulty since it copies the output id's within the input_ids)
5. https://drlee.io/step-by-step-guide-fine-tuning-metas-llama-3-2-1b-model-f1262eda36c8
6. https://huggingface.co/blog/ImranzamanML/fine-tuning-1b-llama-32-a-comprehensive-article
7. https://blog.futuresmart.ai/fine-tune-llama-32-vision-language-model-on-custom-datasets
8. https://github.com/meta-llama/llama-cookbook/blob/main/getting-started/finetuning/multigpu_finetuning.md



### Reading the Counsel Chat Dataset

In [4]:
with open('processed_data/counselchat_top_votes_train.pkl', 'rb') as file:
    dataset_top_votes_train = pickle.load(file)

dataset_top_votes_train.head()

,topic,question,answerText
0,relationships,I am currently suffering from erectile dysfunc...,"Hi, First and foremost, I want to acknowledge ..."
1,family-conflict,For the past week or so me and my boyfriend ha...,Forgetting one's emotions is impossible.Since ...
2,depression,I am in high school and have been facing anxie...,"Hi Helena,I felt a bit sad when I read this. T..."
3,anxiety,I'm concerned about my boyfriend. I suffer fro...,Hello! Thank you for your question. There are ...
4,spirituality,"I'm a Christian teenage girl, and I have lost ...",Having sex with your boyfriend is and was a mi...


In [5]:
with open('processed_data/counselchat_top_votes_test.pkl', 'rb') as file:
    dataset_top_votes_test = pickle.load(file)

dataset_top_votes_test.head()

,topic,question,answerText
0,relationships,I had to go to the emergency room today to get...,It is extremely frustrating when our significa...
1,marriage,What makes a healthy marriage last? What makes...,"This is a fantastic question. In one sentence,..."
2,relationships,"I'm a female freshman in high school, and this...","First off, I think it is great that you are wi..."
3,intimacy,"My wife and I are newly married, about 2 month...","You are newly married, you Have a hectic sched..."
4,legal-regulatory,"I think I have depression, anxiety, bipolar di...",It can be difficult to get counseling if you d...


Creating the HuggingFace dataset using Pandas Dataframe

In [6]:
dataset_train = Dataset.from_pandas(dataset_top_votes_train)
dataset_test = Dataset.from_pandas(dataset_top_votes_test)

In [7]:
dataset = DatasetDict()
dataset['train'] = dataset_train
dataset['test'] = dataset_test

In [8]:
dataset

DatasetDict({
    train: Dataset({
        features: ['topic', 'question', 'answerText'],
        num_rows: 690
    })
    test: Dataset({
        features: ['topic', 'question', 'answerText'],
        num_rows: 173
    })
})

### Fine-Tuning Code

#### Loading the model and tokenizer

In [ ]:
max_seq_length = 2048 
dtype = None # None for auto-detection.
load_in_4bit = False # Use 4bit quantization to reduce memory usage. Can be False.

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Llama-3.2-1B-Instruct",
    max_seq_length = max_seq_length,
    load_in_4bit=load_in_4bit,
    dtype=dtype,
    device_map="auto"
)

#### Setting up the PEFT settings for the model

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth", 
    random_state = 42,
    use_rslora = False,
    loftq_config = None,
)

#### Forming the chat template

In [ ]:
# Define a function to apply the chat template
def format_chat_template(example):
        
    messages = [
        {"role": "system", "content": "You are an expert mental health professional trained to counsel and guide patients suffering from ill mental-health"},
        {"role": "user", "content": example['question']},
        {"role": "assistant", "content": example['answerText']}
    ]
    
    prompt = tokenizer.apply_chat_template(messages, tokenize=False)

    return {"text": prompt}

In [ ]:
dataset_formatted = dataset.map(format_chat_template)

In [ ]:
print(dataset_formatted['train']['text'][0])

#### Initializing the TRL SFTTrainer and related Arguments

In [ ]:
new_model = "./llama32-sft-fine-tune-counselchat"

training_args = TrainingArguments(
        output_dir=new_model,
        per_device_train_batch_size=2,
        per_device_eval_batch_size=2,
        # gradient_accumulation_steps=4,
        eval_strategy="steps",
        eval_steps=0.1,
        logging_strategy="steps",
        logging_steps=50,
        save_strategy="epoch",
        warmup_steps = 5,
        num_train_epochs = 3,
        learning_rate = 1e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 42,
        report_to = "none",
    )

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset=dataset_formatted["train"],
    eval_dataset=dataset_formatted["test"],
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    data_collator = DataCollatorForSeq2Seq(tokenizer = tokenizer), #only use when using train_on_responses_only()
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = training_args)

In [ ]:
trainer.train_dataset

In [ ]:
print(tokenizer.decode(trainer.train_dataset['input_ids'][0]))

#### Only Focus on the `Response Part` for the generation

In [ ]:
trainer = train_on_responses_only(
    trainer,
    instruction_part = "<|start_header_id|>user<|end_header_id|>\n\n",
    response_part = "<|start_header_id|>assistant<|end_header_id|>\n\n",
)

In [ ]:
trainer.train_dataset

In [ ]:
# The labels are created which only contain response. Left Padding is implemented and all the padding tokens are given a score of -100 to avoid loss calculation for pad_tokens
trainer.train_dataset['labels'][0]

#### Train the model

In [ ]:
trainer_stats = trainer.train()

#### Saving the model and tokenizer

Merge the LoRA into the base model and then save 16-bit version

In [ ]:
# model.save_pretrained_merged(new_model, tokenizer, save_method = "merged_16bit")

Just Save LoRA Adapters

In [ ]:
# model.save_pretrained_merged(new_model, tokenizer, save_method = "lora",) #WJust Save LoRA Adapters

# Or run the two below statements
model.save_pretrained(new_model)
tokenizer.save_pretrained(new_model)

### Inference

In [9]:
new_model = "./llama32-sft-fine-tune-counselchat"

In [10]:
max_seq_length = 2048 
dtype = None # None for auto-detection.
load_in_4bit = False # Use 4bit quantization to reduce memory usage. Can be False.

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = new_model,
    max_seq_length = max_seq_length,
    load_in_4bit=load_in_4bit,
    dtype=dtype,
    device_map="auto"
)

==((====))==  Unsloth 2025.3.19: Fast Llama patching. Transformers: 4.50.2.
   \\   /|    NVIDIA RTX A6000. Num GPUs = 1. Max memory: 47.413 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.4.1+cu124. CUDA: 8.6. CUDA Toolkit: 12.4. Triton: 3.0.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.28.post1. FA2 = True]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Unsloth 2025.3.19 patched 16 layers with 16 QKV layers, 16 O layers and 16 MLP layers.


In [13]:
dataset['train']['question'][10]

"My mother has Alzheimer's and she has become so nasty and mean to everyone and she always asks for unrealistic, silly or meaningless items. I get so frustrated and angry, but then I feel guilty because I know it probably isn't her fault. How can I cope with feeling like this? No matter what I do, my mom will almost always find something wrong with it"

In [15]:
for idx in range (0,50):
    
    print(dataset['train']['question'][idx])
    
    FastLanguageModel.for_inference(model)

    messages = [{"role": "system", "content": "You are an expert mental health professional trained to counsel and guide patients suffering from ill mental-health"},
        {"role": "user", "content": dataset['train']['question'][idx]}]

    prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
        
    inputs = tokenizer(prompt, return_tensors='pt', padding=True, truncation=True).to("cuda")

    outputs = model.generate(**inputs, max_new_tokens=2048, num_return_sequences=1)

    text = tokenizer.decode(outputs[0], skip_special_tokens=True)

    print(text.split("assistant")[1])
    
    print('---------------------------------------------------')

I am currently suffering from erectile dysfunction and have tried Viagra, Cialis, etc. Nothing seemed to work. My girlfriend of 3 years is very sexually frustrated. I told her that it is okay for her to have sex with other men. Is that really okay? Is it okay for my girlfriend to have sex with other men since I can't sexually perform?


Hi, It sounds like you are very frustrated with your own performance and your girlfriend's desire to have sex with other men. First, let me say that it is completely understandable that you would be frustrated. You are not alone in this, and many people feel the same way. I would suggest that you take a step back and try to understand where your frustration comes from. Are you feeling rejected? Are you feeling angry? Are you feeling sad? If you are feeling rejected, then this may be a reason to talk with your girlfriend about this. If you are feeling angry, then you may want to talk with your girlfriend about this. If you are feeling sad, then you may w